In [14]:
import openai
import json
import pg8000
from dotenv import load_dotenv
import os
import uuid


# Load environment variables
load_dotenv(dotenv_path="./.env")

# OpenAI API Key
openai.api_key = os.getenv("OPENAI_API_KEY")

# Database connection parameters
DATABASE_URL = os.getenv("POSTGRES_URL")
DATABASE_USER = os.getenv("POSTGRES_USER")
DATABASE_PASS = os.getenv("POSTGRES_PASSWORD")
DATABASE_DB = os.getenv("POSTGRES_DATABASE")
DATABASE_HOST = os.getenv("POSTGRES_HOST")


### Start with Connecting to DB, also functions for receiving from DB

In [17]:
def get_db_connection():
    """ Creates a Database connection """
    conn = pg8000.connect(
        user=DATABASE_USER,
        password=DATABASE_PASS,
        host=DATABASE_HOST,
        database=DATABASE_DB
    )
    return conn


def get_user_diet(uid, cursor):
    """
    Retrieves the User Diet Restrictions
    
    Inputs:
    uid - user ID (string or UUID object)
    cursor - database cursor object

    Returns:
    String of all associated dietary requirements
    """
    try:
        # Ensure the UID is a UUID object
        if isinstance(uid, str):
            uid = uuid.UUID(uid)

        query = """
            SELECT dt."dietName"
            FROM userpreference as up
            JOIN diettype as dt ON up."dietId" = dt."dietId" 
            WHERE up.uid = %s;
        """
        cursor.execute(query, (str(uid),))  # Use parameterized queries
        diets = cursor.fetchall()
        diet_list = [diet[0] for diet in diets]
        return ', '.join(diet_list)
    except Exception as e:
        print(f"Database error: {e}")
        return ""
    

def get_conversation_context(uid, cursor):
    """ 
    Gets context to give to chatbot 
    
    Inputs:
    uid - user Id (UUID as string or object)
    cursor - database cursor object

    Returns:
    List of 10 most previous questions and answers formatted to give
    chatbot correct context
    """

    try:
        # Ensure the UID is a UUID object
        if isinstance(uid, str):
            uid = uuid.UUID(uid)

        query = """
            SELECT "userQuestion", "chatResponse"
            FROM context
            WHERE uid = %s AND "recipeId" IS NULL
            ORDER BY timestamp DESC
            LIMIT 10;
        """
        cursor.execute(query, (str(uid),))  # Parameterized query for safety
        context = cursor.fetchall()

        # Flatten the context into a list of messages
        messages = []
        for user_q, chat_resp in reversed(context):
            messages.append({"role": "user", "content": user_q})
            messages.append({"role": "assistant", "content": chat_resp})
        return messages
    except Exception as e:
        print(f"Database error in get_conversation_context: {e}")
        return []
    

def get_user_recipes(uid, cursor):
    """
    Retrieves the previous user recipes
    
    Inputs:
    uid - user Id (UUID as string or object)
    cursor - database cursor object

    Returns:
    List of previous recipes names for context
    """

    try:
        # Ensure the UID is a UUID object
        if isinstance(uid, str):
            uid = uuid.UUID(uid)

        query = """
            SELECT "recipeName"
            FROM recipe
            WHERE uid = %s;
        """
        cursor.execute(query, (str(uid),))  # Parameterized query for safety
        recipes = cursor.fetchall()

        # Extract recipe names
        recipe_names = [recipe[0] for recipe in recipes]
        return recipe_names
    except Exception as e:
        print(f"Database error in get_user_recipes: {e}")
        return []


### Classify the message

In [3]:
def classify_message(user_question, user_diet, context, client):
    classification_rules = """
    You are to classify the user's request into one of the following categories:
    - "recipe": If the user is asking for a recipe.
    - "lifestyle": If the user is seeking lifestyle tips related to their diet.
    - "general": If the user needs clarification about previous prompts or anything that would not be a lifestyle tip.
    - "not applicable": If the request is unrelated to diet or lifestyle.

    Respond with only one of the above categories.
    """

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-mini", 
            messages=[
                {"role": "system", "content": classification_rules},
                {"role": "assistant", "content": f"Dietary preferences: {user_diet}, context: {context}"},
                {"role": "user", "content": user_question}
            ],
            max_tokens=5,
            n=1,
            stop=None,
            temperature=0
        )
        classification = completion.choices[0].message.content.strip().lower()
        return classification
    except Exception as e:
        print(f"Classification error: {e}")
        return None


## Generate a response

#### Store Context

In [4]:
def store_context(userId, user_question, chat_response, classify, class_ID, conn):
    """
    Stores the generated recipe into Postgres Database

    Inputs:\\
    userId - userId for adding to DB\\
    user_question - prompt from the user to add to db\\
    chat_response - formatted response for the context db\\
    classify - the type of chat that was used\\
    class_ID - either a lifestyle tip id or a recipe id\\
    conn - connection to the database
    """
    tip_id = None
    recipe_id = None

    if classify == "recipe":
        recipe_id = class_ID
    elif classify == "lifestyle":
        tip_id = class_ID

    try:
        cursor = conn.cursor()

        query = """
            INSERT INTO context (uid, "tipId", "recipeId", "userQuestion", "chatResponse")
            VALUES (%s, %s, %s, %s, %s)
        """
        cursor.execute(query, (userId, tip_id, recipe_id, user_question, chat_response))

        conn.commit()
        return None
    except Exception as e:
        print(f"Database error: {e}")
        return None

#### Recipe Section

In [5]:
def store_recipe(userId, recipe, conn):
    """
    Stores the generated recipe into Postgres Database

    Inputs:\\
    userId - userId, used for key to store\\
    recipe - new recipe created by chatbot to store to recipe db\\
    conn - connection to the db

    Returns:\\
    recipe_id - id to connect to the context table
    """

    try:
        cursor = conn.cursor()
        
        name = recipe['Recipe']
        ingreds = json.dumps(recipe['Ingredients'])
        prep_time = recipe['Prep Time']
        instruct = json.dumps(recipe['Instructions'])

        query = """
            INSERT INTO recipe (uid, "recipeName", ingredients, instructions, "prepTime")
            VALUES (%s, %s, %s, %s, %s)
            RETURNING "recipeId";
        """
        cursor.execute(query, (userId, name, ingreds, instruct, prep_time))

        recipe_id = cursor.fetchone()[0]
        print(recipe_id)
        conn.commit()
        return recipe_id
    except Exception as e:
        print(f"Database error: {e}")
        return None


def generate_recipe(user_message, user_diet, previous_recipes, client, userId, conn):
    """
    Generates a new recipe for the user

    Inputs:\\
    user_message - prompt from the user\\
    user_diet - dietary restrictions of the user from database\\
    previous_recipes - previous recipes to not repeat, from DB\\
    client - chatGPT client object\\
    userId - userId\\
    conn - connection to database

    Returns:\\
    JSON with the output to return to the user.
    """
    response = client.chat.completions.create(
    model="gpt-4o-mini",  
    messages=[
        {
            "role": "system",
            "content": """You give recipes given certain dietary conditions. 
            You must make sure the recipe is not repeated.
            You must make sure the recipe does not recommend anything the user cannot eat
            with their dietary restrictions"""
        },
        {
            "role": "assistant",
            "content": f"""Dietary restrictions: User has the following diet requirements {user_diet},
                        Previous Recipes: {previous_recipes}"""
        },
        {
            "role": "user",
            "content": user_message
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "recipe_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "Recipe": {
                        "description": "The name of the recipe",
                        "type": "string"
                    },
                    "Ingredients": {
                        "description": "list of ingredients with measurements if needed",
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                    "Prep Time": {
                        "description": "time in minutes to prepare and cook this recipe",
                        "type": "integer"
                    },
                    "Instructions": {
                        "description": "Instructions to create this recipe, stored as JSON",
                        "type": "array",
                        "items": {
                            "type": "string"
                        }
                    },
                    "additionalProperties": False
            }
        }
    }}
    )

    # Print the response
    print(response.choices[0].message.content)
    recipe = json.loads(response.choices[0].message.content)
    class_ID = store_recipe(userId, recipe, conn)

    classify = "recipe"
    chat_response = recipe["Recipe"]

    store_context(userId, user_message, chat_response, classify, class_ID, conn)
    response = {"uid": userId,
                "message": f"Recipe added to Recipe Book: {recipe['Recipe']}",
                "status": "success"}
    return(response)

#### Lifestyle Section

In [6]:
def store_lifestyle_tip(userId, tip, conn):
    """
    Stores the generated Lifestyle Tip into Postgres Database

    Inputs:\\
    userId - userId, used for key to store\\
    tip - new lifestyle tip created by chatbot to store to recipe db\\
    conn - connection to the db

    Returns:\\
    tip_id - id to connect to the context table
    """

    try:
        cursor = conn.cursor()
        
        summary = tip["summary"]
        full_tip = tip["full_tip"]

        query = """
            INSERT INTO tip (uid, summary, tip)
            VALUES (%s, %s, %s)
            RETURNING "tipId";
        """
        cursor.execute(query, (userId, summary, full_tip))

        tip_id = cursor.fetchone()[0]
        print(tip_id)
        conn.commit()
        return tip_id
    except Exception as e:
        print(f"Database error: {e}")
        return None

def generate_lifestyle_tip(user_message, user_diet, context, client, userId, conn):
    """
    Generates a new lifestyle tip for the user

    Inputs:\\
    user_message - prompt from the user\\
    user_diet - dietary restrictions of the user from database\\
    context- context for the chatbot, previous chat messages and answers\\
    client - chatGPT client object\\
    userId - userId\\
    conn - connection to database

    Returns:\\
    JSON with the output to return to the user.
    """
    response = client.chat.completions.create(
    model="gpt-4o-mini",  
    messages=[
        {
            "role": "system",
            "content": """You give lifestyle tips to the user given certain dietary conditions. 
            The lifestyle tips will be relevent to prompt the user supplies.
            The tip will have a very concise summary, then if necessary a longer explanation.
            """
        },
        {
            "role": "assistant",
            "content": f"""Dietary restrictions: User has the following diet requirements {user_diet},
                        Chat Context: {context}"""
        },
        {
            "role": "user",
            "content": user_message
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "tips_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "summary": {
                        "description": "A very concise summary of the tip",
                        "type": "string"
                    },
                    "full_tip": {
                        "description": "A longer explanation of the lifestyle tip",
                        "type": "string"
                    },
                    "additionalProperties": False
            }
        }
    }}
    )

    # Print the response
    print(response.choices[0].message.content)
    tip = json.loads(response.choices[0].message.content)
    class_ID = store_lifestyle_tip(userId, tip, conn)

    classify = "lifestyle"
    chat_response = tip["summary"]
    full_tip = tip["full_tip"]

    store_context(userId, user_message, chat_response, classify, class_ID, conn)
    response = {"uid": userId,
                "message": f"{chat_response} \n\n {full_tip} \n\n Lifestyle tip added to Lifestyle Tips Cards",
                "status": "success"}
    return(response)

#### General Tips

In [7]:
def generate_general(user_message, user_diet, context, client, userId, conn):
    """
    Generates a new general tip for the user

    Inputs:\\
    user_message - prompt from the user\\
    user_diet - dietary restrictions of the user from database\\
    context- context for the chatbot, previous chat messages and answers\\
    client - chatGPT client object\\
    userId - userId\\
    conn - connection to database

    Returns:\\
    JSON with the output to return to the user.
    """
    response = client.chat.completions.create(
    model="gpt-4o-mini",  
    messages=[
        {
            "role": "system",
            "content": """You give general tips to the user given certain dietary conditions. 
            The general tips could just be clarification about previous user questions.
            You will answer the user's prompt
            """
        },
        {
            "role": "assistant",
            "content": f"""Dietary restrictions: User has the following diet requirements {user_diet},
                        Chat Context: {context}"""
        },
        {
            "role": "user",
            "content": user_message
        }
    ],
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "general_schema",
            "schema": {
                "type": "object",
                "properties": {
                    "general_tip": {
                        "description": "Answer to the user's query",
                        "type": "string"
                    },
                    "additionalProperties": False
            }
        }
    }}
    )

    # Print the response
    print(response.choices[0].message.content)
    general_tip = json.loads(response.choices[0].message.content)

    classify = "none"
    chat_response = general_tip["general_tip"]
    class_ID = None

    store_context(userId, user_message, chat_response, classify, class_ID, conn)
    response = {"uid": userId,
                "message": f"{chat_response}",
                "status": "success"}
    return(response)

In [8]:
def generate_response(classification, userId, user_message, user_diet, context, recipes, client, conn):
    """
    Generates a chat response bassed on the previous classification

    Inputs:\\
    classification - based on the classify message function\\
    userId - userId\\
    user_message - prompt from the user\\
    user_diet - dietary restrictions of the user from database\\
    context - context to give Chat, from DB\\
    recipes - previous recipes to not repeat, from DB\\
    client - chatGPT client object\\
    conn - connection to database

    Returns:\\
    JSON with the output to return to the user.
    """

    if classification == "not applicable":
        response = {"uid": userId,
                    "message": "I'm sorry, but I can only assist with dietary and lifestyle-related questions.",
                    "status": "success"}

        return response

    # Adjust the prompt based on classification
    if classification == "recipe":
        response = generate_recipe(user_message, user_diet, recipes, client, userId, conn)
        return response
    elif classification == "lifestyle":
        response = generate_lifestyle_tip(user_message, user_diet, context, client, userId, conn)
        return response
    elif classification == "general":
        response = generate_general(user_message, user_diet, context, client, userId, conn)
        return response
    else:
        response = response = {"uid": userId,
                "message": "I'm sorry, I didn't understand your request.",
                "status": "success"}
        return response


In [18]:
# Connect to Database

conn = get_db_connection()
cursor = conn.cursor()

# Connect to OpenAI
client = openai.OpenAI()

# Example UUID as a string
uid_str = "2a28b491-44d5-40c5-a824-2d9469ddf9d9"

# Convert to a UUID object
uid_obj = uuid.UUID(uid_str)



try:
    user_diet = get_user_diet(uid_obj, cursor)
    context = get_conversation_context(userId, cursor)
    recipes = get_user_recipes(userId, cursor)

    user_message = "Can you give me an example of communicating cleary to a waiter?"

    classify = classify_message(user_message, user_diet, context, client)
    print(classify)
    response = generate_response(classify, userId, user_message, user_diet, context, recipes, client, conn)
    # recipe = generate_recipe(user_message, user_diet, recipes, client, userId, conn)
#     print(response)
    
    # print(response)
    print(user_diet)
    print(context)
    print(recipes)

except Exception as e:
    print("Exception:" + e)

finally:
    conn.close()

general
{"general_tip":"When communicating with a waiter about your dietary restrictions, be clear and concise. For example, you could say, 'Hello, I have some specific dietary needs. I'm on a Low FODMAP and Anti-Inflammatory diet, so I need to avoid certain foods like garlic, onions, and high-FODMAP fruits. Can you please help me find suitable options on the menu?' This statement ensures the waiter understands your restrictions and can assist you effectively."}
LowFodmap, Anti-Inflammatory
[{'role': 'user', 'content': 'Why is ginger recommended for anti-inflammatory diets?'}, {'role': 'assistant', 'content': 'Ginger has anti-inflammatory properties that help reduce inflammation and provide digestive comfort.'}, {'role': 'user', 'content': 'What types of foods are allowed on a Low FODMAP diet?'}, {'role': 'assistant', 'content': 'Low FODMAP diets focus on reducing foods that ferment in the gut, including certain fruits, vegetables, and grains.'}]
['Low FODMAP Roasted Vegetables', 'Low 

In [12]:
recipe

NameError: name 'recipe' is not defined

In [119]:
def store_recipe(userId, recipe, conn):
    """
    Stores the generated recipe into Postgres Database
    """
    try:
        # Use the provided connection to create a cursor
        cursor = conn.cursor()

        # Extract recipe details
        name = recipe['Recipe']
        ingreds = json.dumps(recipe['Ingredients'])
        prep_time = recipe['Prep Time']
        instruct = json.dumps(recipe['Instructions'])

        # Insert query
        query = """
            INSERT INTO recipe (uid, "recipeName", ingredients, instructions, "prepTime")
            VALUES (%s, %s, %s, %s, %s)
            RETURNING "recipeId";
        """
        # Execute the query with the provided parameters
        cursor.execute(query, (userId, name, ingreds, instruct, prep_time))

        # Fetch the auto-generated recipe ID
        recipe_id = cursor.fetchone()[0]

        # Commit the transaction using the connection
        conn.commit()

        print(f"Inserted recipeId: {recipe_id}")
        return recipe_id
    except Exception as e:
        # Roll back the transaction on error
        conn.rollback()
        print(f"Database error: {e}")
        return None

In [120]:
# Connect to Database
conn = get_db_connection()
conn.autocommit = False
store = store_recipe(1, recipe, conn)
print(store)

conn.close()

Inserted recipeId: 30
30


In [112]:
conn = pg8000.connect(
    user=DATABASE_USER,
    password=DATABASE_PASS,
    host=DATABASE_HOST,
    database=DATABASE_DB
)
cursor = conn.cursor()
try:
    # Example query: Fetch User table
    cursor.execute('SELECT * FROM diettype')
    rows = cursor.fetchall()
    print(rows)

    # Example query: Fetch User table
    cursor.execute("""SELECT dt."dietName" FROM userpreference as up
                   JOIN diettype as dt ON up."dietId" = dt."dietId" 
                   WHERE up.uid = 1 """)
    rows = cursor.fetchall()
    print(rows)

    # Fetch column names and their data types for each table
    tables = ['users', 'userpreference', 'diettype', 'context', 'tip', 'recipe']

    for table in tables:
        cursor.execute(f"""
            SELECT column_name, data_type 
            FROM information_schema.columns 
            WHERE table_name = '{table}';
        """)
        columns = cursor.fetchall()
        print(f"Columns and their types in '{table}' table:")
        for column_name, data_type in columns:
            print(f"  {column_name}: {data_type}")
        print()  # Blank line for readability
except Exception as e:
    print("Exception" + e)

    # Close the connection
finally:
    conn.close()

([1, 'GlutenFree'], [2, 'LowFodmap'], [3, 'Mediterranean'], [4, 'DASH'], [5, 'Anti-Inflammatory'], [6, 'LowCal-LowFat'])
(['GlutenFree'], ['Anti-Inflammatory'])
Columns and their types in 'users' table:
  uid: integer
  email: text
  password: text
  username: text

Columns and their types in 'userpreference' table:
  preferenceId: integer
  uid: integer
  dietId: integer

Columns and their types in 'diettype' table:
  dietId: integer
  dietName: text

Columns and their types in 'context' table:
  tipId: integer
  timestamp: timestamp without time zone
  uid: integer
  chatId: integer
  recipeId: integer
  chatResponse: text
  userQuestion: text

Columns and their types in 'tip' table:
  tipId: integer
  uid: integer
  timestamp: timestamp without time zone
  tip: text
  summary: text

Columns and their types in 'recipe' table:
  ingredients: json
  uid: integer
  timestamp: timestamp without time zone
  recipeId: integer
  instructions: json
  prepTime: integer
  recipeName: text

